# Example) Build Generative QA Dataset

# 1. Without Tokenization

1. Create a `SquadGuru` object who is an NLP expert. Let him `gather` the complex squad json dataset organized to make it usable in an NLP task.

   - Constructor Signature

     ```python
     SquadGuru(parser: SquadParser, #parser which implement SquadParser
               tokenizer=None, #tokenizer which implement .tokenize(text: str)
               tags=SQUAD_TAGS, #iterable of str
               versions=SQUAD_VERSIONS #iterable of float
     )
     ```

   - Inject a `parser`, which the guru will use to extract X and Y data from the original suqad dataset.
   - Inject a `tokenizer` that will be used to create tokenized X and Y.
   - Inject an iterable of `tags` that describes the tags of the dataset to load.
   - Inject an iterable of `versions` that describes the versions of the dataset to load. Version's datatype is `float`.


   Here we're gonna use `GenerativeQAParser`. In order to create an instance of it, use static factory method pattern like: `SquadParser.from_nlp_task('GEN_QA')`

In [1]:
from prep_squad.guru import SquadGuru
from prep_squad.parser import SquadParser

squad_parser = SquadParser.from_nlp_task('GEN_QA')
guru = SquadGuru(squad_parser, tags=['dev'], versions=(1.1, 2.0))

2. Use `squadGuru.gather()` to let the guru remember extracted X and Y.

   - Method Signature

     ```python
     squadGuru.gather(only_first_answer=False, 
                      verbose=False)
```
     
     - Set `only_first_answer` to extract the first answer in each of question-answers sets.
     - Set `verbose` to print some logs.

In [2]:
guru.gather(only_first_answer=True, verbose=True)

SQuAD-v1.1 dev dataset has been parsed.
SQuAD-v2.0 dev dataset has been parsed.


## Get DataFrame Table

In [3]:
guru.to_dataframe()

,Input,Target
0,Super Bowl 50 was an American football game to...,Denver Broncos
1,Super Bowl 50 was an American football game to...,Carolina Panthers
2,Super Bowl 50 was an American football game to...,"Santa Clara, California"
3,Super Bowl 50 was an American football game to...,Denver Broncos
4,Super Bowl 50 was an American football game to...,gold
...,...,...
16493,"The pound-force has a metric counterpart, less...",kilogram-force
16494,"The pound-force has a metric counterpart, less...",kilopond
16495,"The pound-force has a metric counterpart, less...",slug
16496,"The pound-force has a metric counterpart, less...",kip


## Get Numpy Array 

In [4]:
guru.to_numpy()

array([['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. [SEP] Which NFL team represented the AFC at Super Bowl 50?',
        'Denver Broncos'],
       ['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2

## Save to files

In [5]:
#passage-question
#answer
guru.to_file('examples/genqa.pq.example.txt', 'examples/genqa.a.example.txt')

# 2. Applying Bert Tokenization

1. Create a `SquadGuru` object who is an NLP expert. Here we give him a pretrained `BertTokenizer`.

In [6]:
from prep_squad.guru import SquadGuru
from prep_squad.parser import SquadParser
from transformers import BertTokenizer

task = SquadParser.from_nlp_task('GEN_QA')
tok = BertTokenizer.from_pretrained('bert-large-cased')
guru = SquadGuru(task, tok, tags=['dev'], versions=(1.1, 2.0))

In [7]:
guru.gather(only_first_answer=True, verbose=True)

SQuAD-v1.1 dev dataset has been parsed.
SQuAD-v2.0 dev dataset has been parsed.


In [8]:
guru.to_dataframe()

,Input,Target
0,Super Bowl 50 was an American football game to...,Denver Broncos
1,Super Bowl 50 was an American football game to...,Carolina Panthers
2,Super Bowl 50 was an American football game to...,"Santa Clara , California"
3,Super Bowl 50 was an American football game to...,Denver Broncos
4,Super Bowl 50 was an American football game to...,gold
...,...,...
16493,"The pound - force has a metric counterpart , l...",k ##ilo ##gram - force
16494,"The pound - force has a metric counterpart , l...",k ##ilo ##po ##nd
16495,"The pound - force has a metric counterpart , l...",s ##lug
16496,"The pound - force has a metric counterpart , l...",k ##ip


In [9]:
guru.to_numpy()

array([['Super Bowl 50 was an American football game to determine the champion of the National Football League ( NFL ) for the 2015 season . The American Football Conference ( AFC ) champion Denver Broncos defeated the National Football Conference ( NFC ) champion Carolina Panthers 24 – 10 to earn their third Super Bowl title . The game was played on February 7 , 2016 , at Levi \' s Stadium in the San Francisco Bay Area at Santa Clara , California . As this was the 50th Super Bowl , the league emphasized the " golden anniversary " with various gold - themed initiatives , as well as temporarily su ##sp ##ending the tradition of naming each Super Bowl game with Roman n ##ume ##rals ( under which the game would have been known as " Super Bowl L " ) , so that the logo could prominently feature the Arabic n ##ume ##rals 50 . [SEP] Which NFL team represented the AFC at Super Bowl 50 ?',
        'Denver Broncos'],
       ['Super Bowl 50 was an American football game to determine the champion 

In [10]:
#passage-question
#answer
guru.to_file('examples/genqa.pq.tok.example.txt', 'examples/genqa.a.tok.example.txt')